# CRIS — Exploratory Data Analysis (EDA)

This notebook explores the raw data loaded into SQL Server,
validates distributions, and generates visualizations for business understanding.

**Prerequisites:** Run `python src/data_ingestion.py` first to populate the database.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.db_connector import run_query

# Style
sns.set_theme(style='whitegrid', palette='viridis')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print('Libraries loaded.')

## 1. Data Overview

In [ ]:
# Load tables
customers = run_query('SELECT * FROM dbo.customers')
orders = run_query('SELECT * FROM dbo.orders')
items = run_query('SELECT * FROM dbo.order_items')

print(f'Customers: {len(customers)} rows')
print(f'Orders:    {len(orders)} rows')
print(f'Items:     {len(items)} rows')
print()
customers.head()

In [ ]:
orders.head()

In [ ]:
items.head()

## 2. Churn Distribution

In [ ]:
labels = run_query('SELECT * FROM dbo.customer_labels')
print(f'Labeled customers: {len(labels)}')
print(f'\nChurn distribution:')
print(labels['is_churned'].value_counts())
print(f'\nChurn rate: {labels["is_churned"].mean():.2%}')

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
colors = ['#2ecc71', '#e74c3c']
labels['is_churned'].value_counts().plot.pie(
    labels=['Safe (0)', 'Churned (1)'],
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    ax=ax,
    textprops={'fontsize': 14}
)
ax.set_ylabel('')
ax.set_title('Customer Churn Distribution', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. RFM Distributions

In [ ]:
features = run_query('SELECT * FROM dbo.customer_features')
# Merge with labels
df = features.merge(labels[['customer_id', 'is_churned']], on='customer_id')

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, col in enumerate(['recency_days', 'frequency', 'monetary']):
    ax = axes[i]
    for label, color in zip([0, 1], ['#2ecc71', '#e74c3c']):
        subset = df[df['is_churned'] == label]
        ax.hist(subset[col], bins=20, alpha=0.6, color=color,
                label='Safe' if label == 0 else 'Churned')
    ax.set_title(col.replace('_', ' ').title(), fontsize=14, fontweight='bold')
    ax.set_xlabel(col)
    ax.legend()

plt.suptitle('RFM Distributions by Churn Status', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 4. Feature Correlation Heatmap

In [ ]:
from src.preprocessing import FEATURE_COLUMNS

corr_matrix = df[FEATURE_COLUMNS + ['is_churned']].corr()

fig, ax = plt.subplots(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    square=True,
    linewidths=0.5,
    ax=ax
)
ax.set_title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Order Trends Over Time

In [ ]:
orders['order_date'] = pd.to_datetime(orders['order_date'])
orders['month'] = orders['order_date'].dt.to_period('M')

monthly = orders.groupby('month').agg(
    order_count=('order_id', 'count'),
    revenue=('total_amount', 'sum'),
    unique_customers=('customer_id', 'nunique')
).reset_index()
monthly['month'] = monthly['month'].astype(str)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].bar(monthly['month'], monthly['order_count'], color='#3498db')
axes[0].set_title('Monthly Order Count', fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

axes[1].bar(monthly['month'], monthly['revenue'], color='#e67e22')
axes[1].set_title('Monthly Revenue', fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

axes[2].bar(monthly['month'], monthly['unique_customers'], color='#9b59b6')
axes[2].set_title('Monthly Active Customers', fontweight='bold')
axes[2].tick_params(axis='x', rotation=45)

# Draw observation/prediction window boundary
for ax in axes:
    ax.axvline(x=5.5, color='red', linestyle='--', linewidth=2, label='Window Cutoff')
    ax.legend()

plt.suptitle('Time-Series Trends (Red = Observation/Prediction Boundary)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. Regional & Segment Analysis

In [ ]:
region_churn = labels.groupby('region')['is_churned'].mean().sort_values(ascending=False)
segment_churn = labels.groupby('segment')['is_churned'].mean().sort_values(ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

region_churn.plot.bar(ax=axes[0], color='#e74c3c', alpha=0.8)
axes[0].set_title('Churn Rate by Region', fontweight='bold')
axes[0].set_ylabel('Churn Rate')
axes[0].tick_params(axis='x', rotation=0)

segment_churn.plot.bar(ax=axes[1], color='#e67e22', alpha=0.8)
axes[1].set_title('Churn Rate by Segment', fontweight='bold')
axes[1].set_ylabel('Churn Rate')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 7. Key Takeaways

- **Class Distribution**: Document the churn rate and imbalance ratio
- **RFM Signals**: Higher recency (more days since last order) correlates with churn
- **Temporal Patterns**: Order volume may decline for churned customers in later observation months
- **Feature Correlations**: Identify which features are most correlated with churn